In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import sys
import pickle
import csv
import random

In [ ]:
#drug_iddict        final_Virus_DTI_add_organism.tsv  human_seq_dict.pkl    VDTI_net.npy           virusseq_add_ncov.pkl
#drug_info.tsv      HDTI_net.npy                      human_seq_iddict.pkl  VHI_net.npy            virusseq_add_ncovseqdict.pkl
#Drug_simi_net.npy  human.npy                         PPI_net.npy           virusseq_add_ncov.npy

In [ ]:
#NPY
drug_simi_net = np.load("DTI-data/Drug_simi_net.npy")
print(drug_simi_net.shape)
print(drug_simi_net[0])
HDTI_net = np.load("DTI-data/HDTI_net.npy")
print(HDTI_net.shape)
print(HDTI_net[0])
human = np.load("DTI-data/human.npy")
print(human.shape)
print(human[0])
ppi_net = np.load("DTI-data/PPI_net.npy")
print(ppi_net.shape)
print(ppi_net[0])
#This is our ultimate target
vdti_net = np.load("DTI-data/VDTI_net.npy")
print("!", vdti_net.shape)
print("!", vdti_net[0])
vhi_net = np.load("DTI-data/VHI_net.npy")
print(vhi_net.shape)
print(vhi_net[0])
virusseq_add_ncov = np.load("DTI-data/virusseq_add_ncov.npy")
print(virusseq_add_ncov.shape)
print(virusseq_add_ncov[0])

In [ ]:

#PICKLE
with open("DTI-data/human_seq_dict.pkl", 'rb') as f:
    human_seq_dict = pickle.load(f)
with open("DTI-data/human_seq_iddict.pkl", 'rb') as f:    
    human_seq_iddict = pickle.load(f)
with open("DTI-data/virusseq_add_ncov.pkl", 'rb') as f:
    virusseq_add_ncov_dict = pickle.load(f)
with open("DTI-data/virusseq_add_ncovseqdict.pkl", 'rb') as f:
    virusseq_add_ncovseqdict = pickle.load(f)
with open("DTI-data/drug_iddict", 'rb') as f:
    drug_iddict = pickle.load(f)

print(human_seq_dict[list(human_seq_dict.keys())[0]][:100], "\n")
print(human_seq_iddict[list(human_seq_iddict.keys())[0]], "\n")
print(virusseq_add_ncov_dict[list(virusseq_add_ncov_dict.keys())[0]], "\n")
print(virusseq_add_ncovseqdict[list(virusseq_add_ncovseqdict.keys())[0]][:100], "\n")
print(drug_iddict[list(drug_iddict.keys())[0]], "\n")

In [ ]:

#TSV
with open("DTI-data/drug_info.tsv", "r") as f:
    drug_info = csv.reader(f, delimiter = '\t')
    drug_info = list(drug_info)
with open("DTI-data/final_Virus_DTI_add_organism.tsv", "r") as f:
    final_Virus_DTI_add_organism = csv.reader(f, delimiter = '\t') 
    final_Virus_DTI_add_organism = list(final_Virus_DTI_add_organism)

print(drug_info[0:2])
print("")
print(final_Virus_DTI_add_organism[0:2])

In [ ]:
!ls

In [ ]:
# KnowledgeGraphEmbedding expects 5 files

# entities.dict - maps entities to ids
# relations.dict - maps relations to ids
# train.txt - entity id to relation id
# valid.txt - can be blank
# test.txt

print(HDTI_net.shape)
print(HDTI_net[0])

print("!", vdti_net.shape)
print("!", vdti_net[0])

print(vhi_net.shape)
print(vhi_net[0])

#entities.dict

DRUG_COUNT = 6255
HUMAN_COUNT = 2567
VIRUS_COUNT = 404

sum_ = DRUG_COUNT + HUMAN_COUNT + VIRUS_COUNT

HUMAN_OFFSET = 6255
VIRUS_OFFSET = 6255 + 2567

names = ["vdti", "hdti", "vhi", "ppi", "drug_s", "human_s", "virus_s"]
vars = [vdti_net, HDTI_net, vhi_net, ppi_net, drug_simi_net, human, virusseq_add_ncov]
lens = [len(arr) for arr in vars]
#lens = [vdti_net.shape[0], HDTI_net.shape[0], vhi_net.shape[0], ppi_net.shape[0], drug_simi_net.shape[0], human_seq_dict.shape[0], virusseq_add_ncov.shape[0]]
run_sum = [sum(lens[:i]) for i in range(1, len(names) + 1)]
offset = {names[i]:run_sum[i] for i, _ in enumerate(names)}

with open("KnowledgeGraphEmbedding/data/C-19/entities.dict", "w") as f:
    for i in range(DRUG_COUNT + HUMAN_COUNT + VIRUS_COUNT):
        f.write(str(i) + "\t" + str(i) + "\n")

#relations.dict

with open("KnowledgeGraphEmbedding/data/C-19/relations.dict", "w") as f:
    f.write("0\t0\n")
    f.write("1\t1\n")
    f.write("2\t2\n")
    f.write("3\t3\n")
    f.write("4\t4\n")
    f.write("5\t5\n")
    f.write("6\t6")
#with open("KnowledgeGraphEmbedding/data/C-19/entities.dict" as f):

#all.txt

HIGH_SIM_THRES = 0.50

with open("KnowledgeGraphEmbedding/data/C-19/all.dict", "w") as f:
    #drug -> human
    for (i, j), v in np.ndenumerate(HDTI_net):
        assert(j + HUMAN_OFFSET < sum_)
        if v:
            f.write(str(i) + "\t" + "0"  + "\t" + str(j + HUMAN_OFFSET) + "\n")

    #drug -> virus
    i = 0
    for (i, j), v in np.ndenumerate(vdti_net):
        if v:
            f.write(str(i) + "\t" + "1"  + "\t" +  str(j + VIRUS_OFFSET) + "\n")  

    #virus -> human
    i = 0
    for (i, j), v in np.ndenumerate(vhi_net):
        assert(i + VIRUS_OFFSET < sum_)
        assert(j + HUMAN_OFFSET < sum_)
        if v:
            f.write(str(i + VIRUS_OFFSET) + "\t" + "2"  + "\t" + str(j + HUMAN_OFFSET) + "\n")          

    for (i, j), v in np.ndenumerate(ppi_net):

        if v:
            f.write(str(i + HUMAN_OFFSET) + "\t" + "3"  + "\t" + str(j + HUMAN_OFFSET) + "\n")


    for (i, j), v in np.ndenumerate(drug_simi_net):
        if v >= HIGH_SIM_THRES:
            f.write(str(i) + "\t" + "4"  + "\t" + str(j) + "\n")


    for (i, j), v in np.ndenumerate(human):
        if v >= HIGH_SIM_THRES:
            f.write(str(i + HUMAN_OFFSET) + "\t" + "5"  + "\t" + str(j + HUMAN_OFFSET) + "\n")


    for (i, j), v in np.ndenumerate(virusseq_add_ncov):
        if v >= HIGH_SIM_THRES:
            f.write(str(i + VIRUS_OFFSET) + "\t" + "6"  + "\t" + str(j + VIRUS_OFFSET) + "\n")


TRAIN_TEST_SPLIT = 0.85
TRAIN_VAL_SPLIT  = 0.90

print("start write")

with open("KnowledgeGraphEmbedding/data/C-19/all.dict", "r") as f:
    lst = f.readlines()
    all_drug_indices = [i for i in range(DRUG_COUNT)]
    train_t_drug_indices = random.sample(all_drug_indices, int(len(all_drug_indices) * TRAIN_TEST_SPLIT))
    train_drug_indices   = random.sample(train_t_drug_indices, int(len(train_t_drug_indices) * TRAIN_VAL_SPLIT)) 
    #train_t = random.sample(lst, int(len(lst) * TRAIN_TEST_SPLIT))
    #train   = random.sample(train_t, int(len(train_t) * TRAIN_VAL_SPLIT))
    val_drug_indices  = [e for e in train_t_drug_indices if e not in train_drug_indices]
    test_drug_indices = [e for e in all_drug_indices if e not in train_t_drug_indices]
    assert(len(val_drug_indices) > 0 and len(test_drug_indices) > 0)
    #train   = [e for e in lst if e.split("\t")[1] in train_drug_indices]
    #val     = [e for e in lst if e.split("\t")[1] in val_drug_indices]
    #test    = [e for e in lst if e.split("\t")[1] in test_drug_indices]
    #assert disjoint
    pass
    print(lst[0].split("\t"))
    print(lst[1].split("\t")[2].strip("\n"))
    test  = [e for e in lst if  int(e.split("\t")[0].strip("\n")) in test_drug_indices
                            and e.split("\t")[1] in ["0", "1"]]
    val   = [e for e in lst if  int(e.split("\t")[0].strip("\n")) in val_drug_indices
                            and e.split("\t")[1] in ["0", "1"]]
    #train = [e for e in lst if  int(e.split("\t")[0].strip("\n")) not in (test_drug_indices + val_drug_indices)
    #                        or  e.split("\t")[1]  not in ["0", "1"]]
    train = [e for e in lst if e not in test and e not in val]
    for e in [test, val, train]:
        assert(len(e) > 0)
    #train.txt
    with open("KnowledgeGraphEmbedding/data/C-19/train.txt", "w+") as f:
        f.writelines(train)
    #valid.txt
    with open("KnowledgeGraphEmbedding/data/C-19/valid.txt", "w+") as f:
        f.writelines(val)
    #test.txt
    with open("KnowledgeGraphEmbedding/data/C-19/test.txt",  "w+") as f:
        f.writelines(test)

print("done")

In [ ]:
#train, test
# -lr 0.0001 --max_steps 150000 \
!CUDA_LAUNCH_BLOCKING=1 CUDA_VISIBLE_DEVICES=0 python -u KnowledgeGraphEmbedding/codes/run.py --do_train \
 --cuda \
 --do_valid \
 --do_test \
 --data_path KnowledgeGraphEmbedding/data/C-19 \
 --model RotatE \
 -n 256 -b 1024 -d 1000 \
 -g 24.0 -a 1.0 -adv \
 -lr 0.0001 --max_steps 1000 \
 --n_targets 404 \
 -save KnowledgeGraphEmbedding/models/RotatE_C-19_0 --test_batch_size 8 -de